In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# Define the neural network
class ODENet(tf.keras.Model):
    def __init__(self):
        super(ODENet, self).__init__()
        self.dense1 = tf.keras.layers.Dense(100, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense2 = tf.keras.layers.Dense(100, activation='tanh', kernel_regularizer=tf.keras.regularizers.l2(0.01))
        self.dense3 = tf.keras.layers.Dense(1, activation='linear')

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return self.dense3(x)

model = ODENet()

In [3]:
# Central difference for second derivative
def second_derivative(y, h=0.01):
    d2y = (y[2:] - 2*y[1:-1] + y[:-2]) / h**2
    return d2y

# Loss function
def loss_fn(model, x, f, h=0.01):
    with tf.GradientTape(persistent=True) as tape2:
        with tf.GradientTape(persistent=True) as tape1:
            tape2.watch(x)
            tape1.watch(x)
            y_pred = model(x)
        dy_dx = tape1.gradient(y_pred, x)
    d2y_dx2 = tape2.gradient(dy_dx, x)

    # Residual from the differential equation
    residual = (d2y_dx2 - f(x)) / h

    # Calculate the loss as the squared error
    loss = tf.reduce_mean(tf.square(residual))
    
    # Add boundary conditions to the loss
    loss += 0.5 * tf.square(model(tf.constant([[0.0]], dtype=tf.float32)))
    loss += 0.5 * tf.square(model(tf.constant([[1.0]], dtype=tf.float32)))
    
    return loss


In [4]:
# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adamax(learning_rate=0.001)

# Train the model
def train_step(model, x, f, h=0.01):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, x, f, h)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

# Define f(x)
def f(x):
    return model(x)

# Define the analytical solution
def analytical_solution(t):
    return (-np.exp(1)/(-np.exp(2)+1)*np.exp(t) + np.exp(1)/(-np.exp(2)+1)*np.exp(-t))


# Boundary conditions
y0 = tf.constant([[0.0]], dtype=tf.float32)
y1 = tf.constant([[1.0]], dtype=tf.float32)

# Train the model